In [19]:
import os
import evdev
from threading import Lock

In [12]:
devices = [evdev.InputDevice(path) for path in evdev.list_devices()]

In [13]:
devices

[InputDevice('/dev/input/event20'), InputDevice('/dev/input/event7')]

In [14]:
for device in devices:
    print(device.path, device.name, device.phys)

/dev/input/event20 Logitech G29 Driving Force Racing Wheel usb-0000:00:14.0-2/input0
/dev/input/event7 LiteOn Lenovo Calliope USB Keyboard Consumer Control usb-0000:00:14.0-3/input1


In [15]:
import evdev
from evdev import ecodes, InputDevice

device = evdev.list_devices()[0]
evtdev = InputDevice(device)
val = 65535  # val \in [0,65535]
evtdev.write(ecodes.EV_FF, ecodes.FF_RUMBLE, val)

In [16]:
import time
import evdev
from evdev import ecodes, ff, InputDevice

device = evdev.list_devices()[0]
evtdev = InputDevice(device)

In [26]:
class WheelControlVibration():
    """
    This class control the Logitech G29 steering wheel vibration.
    It waits for a key event and triggers vibration as long as key is pressed.

    Requirements:
    - evdev: For interacting with input devices.
    - os: For interacting with the file system (raw HID access).
    - time: For controlling vibration duration.
    - threading.Lock: To ensure thread-safe access to the raw device.
    - pygame:  For event handling and joystick input (if needed for key presses).
    """

    # HIDRAW_DEVICE = "hidraw7"  #logitech_raw

    def __init__(self, device):
        """
        Initializes the WheelControlVibration object. Opens the raw
        HID device for the Logitech G29.
        """
        self.raw_dev = None
        try:
            self.raw_dev = os.open(device, os.O_RDWR)
            print("Logitech G29 initialized successfully.")
        except OSError as e:
            print(
                f"Failed to initialize Logitech G29 device {device}. Vibration will not work.  Error: {e}"
            )
            self.raw_dev = None  # Important: Set to None to prevent errors later.

        self._steering_wheel_write_lock = Lock()
        self.vibration_active = False  # Track if vibration is active

    def vibrate(self, duration=10, intensity=25):
        """
        Triggers the vibration of the Logitech G29 steering wheel.

        Args:
            duration (float): The duration of the vibration in seconds. Increase value for continuous.
            intensity (int): The intensity of the vibration (0-127). I have set the Default to 50.
        """
        if self.raw_dev is None:
            print("Raw device not initialized.  Cannot vibrate.")
            return

        # Clamp intensity to the valid range.
        intensity = max(0, min(intensity, 60))

        # Vibrate uses slot F1.
        force_altitude = intensity  # Use intensity directly.

        with self._steering_wheel_write_lock:
            try:
                if duration > 0 and not self.vibration_active:
                    # Start vibration.
                    os.write(
                        self.raw_dev,
                        bytearray(
                            [
                                0x21,
                                0x06,
                                128 + force_altitude,
                                128 - force_altitude,
                                8,
                                8,
                                0x0F,
                            ]
                        ),
                    )
                    self.vibration_active = True
                elif duration == 0 and self.vibration_active:
                    os.write(
                        self.raw_dev,
                        bytearray([0x23, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00]),
                    )  # Stop vibration
                    self.vibration_active = False
            except OSError as e:
                print(f"Error writing to raw device to vibrate: {e}")

    def close(self):
        """
        Closes the connection to the Logitech Steering Wheel device.
        """
        if self.raw_dev:
            try:
                os.close(self.raw_dev)
                self.raw_dev = None
                print("Raw device closed.")
            except OSError as e:
                print(f"Error closing raw device: {e}")

    def __del__(self):
        """
        Destructor to ensure devices is closed
        """
        self.close()

In [27]:
model = WheelControlVibration("/dev/input/event20")

Logitech G29 initialized successfully.
Raw device closed.


In [28]:
model.vibrate()

Error writing to raw device to vibrate: [Errno 22] Invalid argument


In [45]:
import os
import time
import evdev
from threading import Lock


class WheelControlVibration:
    """
    This class control the Logitech G29 steering wheel vibration.
    It waits for a key event and triggers vibration as long as key is pressed.

    Requirements:
    - evdev: For interacting with input devices.
    - os: For interacting with the file system (raw HID access).
    - time: For controlling vibration duration.
    - threading.Lock: To ensure thread-safe access to the raw device.
    - pygame:  For event handling and joystick input (if needed for key presses).
    """

    # We use the symlink created by the udev rule
    # KERNEL=="hidraw*", ATTRS{idVendor}=="046d", ATTRS{idProduct}=="c24f", MODE="0666", SYMLINK+="logitech_raw"
    # This symlink will point to the correct hidraw device (e.g., /dev/hidraw7)
    HIDRAW_DEVICE = "/dev/logitech_raw"

    def __init__(self, device):
        """
        Initializes the WheelControlVibration object. Opens the raw
        HID device for the Logitech G29.
        """
        self.raw_dev = None
        try:
            # Use the correct raw device path for force feedback commands.
            self.raw_dev = os.open(device, os.O_RDWR)
            print(f"Logitech G29 initialized successfully using {device}.")
        except OSError as e:
            print(
                f"Failed to initialize Logitech G29 device {device}. Vibration will not work. Error: {e}"
            )
            self.raw_dev = None  # Important: Set to None to prevent errors later.

        self._steering_wheel_write_lock = Lock()
        self.vibration_active = False  # Track if vibration is active

    def vibrate(self, duration=0.05, intensity=25):
        """
        Triggers the vibration of the Logitech G29 steering wheel.

        Args:
            duration (float): The duration of the vibration in seconds. Increase value for continuous.
            intensity (int): The intensity of the vibration (0-127). I have set the Default to 50.
        """
        if self.raw_dev is None:
            print("Raw device not initialized. Cannot vibrate.")
            return

        # Clamp intensity to the valid range.
        intensity = max(0, min(intensity, 60))

        # Vibrate uses slot F1.
        force_altitude = intensity  # Use intensity directly.

        with self._steering_wheel_write_lock:
            try:
                if duration > 0 and not self.vibration_active:
                    # Start vibration.
                    os.write(
                        self.raw_dev,
                        bytearray(
                            [
                                0x21,
                                0x06,
                                128 + force_altitude,
                                128 - force_altitude,
                                8,
                                8,
                                0x0F,
                            ]
                        ),
                    )
                    self.vibration_active = True
                elif duration == 0 and self.vibration_active:
                    os.write(
                        self.raw_dev,
                        bytearray([0x23, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00]),
                    )  # Stop vibration
                    self.vibration_active = False
            except OSError as e:
                print(f"Error writing to raw device to vibrate: {e}")
                # The "invalid argument" error (Errno 22) often occurs here if the
                # device path is incorrect or the packet format is wrong for the interface.

    def close(self):
        """
        Closes the connection to the Logitech Steering Wheel device.
        """
        if self.raw_dev:
            try:
                os.close(self.raw_dev)
                self.raw_dev = None
                print("Raw device closed.")
            except OSError as e:
                print(f"Error closing raw device: {e}")

    def __del__(self):
        """
        Destructor to ensure devices is closed
        """
        self.close()


if __name__ == "__main__":
    # Example usage
    # Use the symlink created by the udev rule instead of a hardcoded evdev path
    model = WheelControlVibration("/dev/hidraw7")
    model.vibrate()
    time.sleep(0.1)
    model.vibrate(duration=0)  # Stop vibration
    model.close()

Logitech G29 initialized successfully using /dev/hidraw7.
Raw device closed.


In [ ]:
class WheelControlVibration:
    """
    This class control the Logitech G29 steering wheel vibration.
    It waits for a key event and triggers vibration as long as key is pressed.

    Requirements:
    - evdev: For interacting with input devices.
    - os: For interacting with the file system (raw HID access).
    - time: For controlling vibration duration.
    - threading.Lock: To ensure thread-safe access to the raw device.
    - pygame:  For event handling and joystick input (if needed for key presses).
    """

    # HIDRAW_DEVICE = "hidraw7"  #logitech_raw

    def __init__(self, device):
        """
        Initializes the WheelControlVibration object. Opens the raw
        HID device for the Logitech G29.
        """
        self.raw_dev = None
        try:
            self.raw_dev = os.open(device, os.O_RDWR)
            print("Logitech G29 initialized successfully.")
        except OSError as e:
            print(
                f"Failed to initialize Logitech G29 device {device}. Vibration will not work.  Error: {e}"
            )
            self.raw_dev = None  # Important: Set to None to prevent errors later.

        self._steering_wheel_write_lock = Lock()
        self.vibration_active = False  # Track if vibration is active

    def vibrate(self, duration=10, intensity=25):
        """
        Triggers the vibration of the Logitech G29 steering wheel.

        Args:
            duration (float): The duration of the vibration in seconds. Increase value for continuous.
            intensity (int): The intensity of the vibration (0-127). I have set the Default to 50.
        """
        if self.raw_dev is None:
            print("Raw device not initialized.  Cannot vibrate.")
            return

        # Clamp intensity to the valid range.
        intensity = max(0, min(intensity, 60))

        # Vibrate uses slot F1.
        force_altitude = intensity  # Use intensity directly.

        with self._steering_wheel_write_lock:
            try:
                if duration > 0 and not self.vibration_active:
                    # Start vibration.
                    os.write(
                        self.raw_dev,
                        bytearray(
                            [
                                0x21,
                                0x06,
                                128 + force_altitude,
                                128 - force_altitude,
                                8,
                                8,
                                0x0F,
                            ]
                        ),
                    )
                    self.vibration_active = True
                elif duration == 0 and self.vibration_active:
                    os.write(
                        self.raw_dev,
                        bytearray([0x23, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00]),
                    )  # Stop vibration
                    self.vibration_active = False
            except OSError as e:
                print(f"Error writing to raw device to vibrate: {e}")

    def close(self):
        """
        Closes the connection to the Logitech Steering Wheel device.
        """
        if self.raw_dev:
            try:
                os.close(self.raw_dev)
                self.raw_dev = None
                print("Raw device closed.")
            except OSError as e:
                print(f"Error closing raw device: {e}")

    def __del__(self):
        """
        Destructor to ensure devices is closed
        """
        self.close()